<a href="https://colab.research.google.com/github/dangni02/Oliveyoung/blob/main/brand%EC%B6%94%EA%B0%80%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
import re
import pandas as pd
import torch
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel

# 1) 데이터 불러오기 & Rating 숫자 추출
file_path = '/content/drive/MyDrive/텍스트마이닝/oliveyoung_sentiment_analysis/oliveyoung_reviews.csv'
df = pd.read_csv(file_path,encoding='cp949',engine='python')
def parse_rating(s):
    m = re.search(r'(\d+)점만점에\s*(\d+)점', str(s))
    return int(m.group(2)) if m else None
df['rating_num'] = df['rating'].apply(parse_rating)
df = df.dropna(subset=['review', 'rating_num'])

# 2) 부정/긍정 리뷰 분리
neg = df[df['rating_num'] <= 2]['review'].tolist()
pos = df[df['rating_num'] >= 4]['review'].tolist()

# 3) KoBERT로 문장 임베딩 생성
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model     = AutoModel.from_pretrained("monologg/kobert")
def embed(texts, batch_size=32):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        toks  = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            out = model(**toks, return_dict=True).last_hidden_state
        # [CLS] 토큰 벡터(0번) 평균 혹은 직접 사용
        cls_emb = out[:,0].cpu().numpy()
        embs.append(cls_emb)
    return np.vstack(embs)

neg_emb = embed(neg)
pos_emb = embed(pos)

# 4) K-means 클러스터링 (클러스터 수 = 3)
kmeans_neg = KMeans(n_clusters=3, random_state=0).fit(neg_emb)
kmeans_pos = KMeans(n_clusters=3, random_state=0).fit(pos_emb)

# 5) 클러스터 레이블 할당 & 결과 확인
clusters = []
for label, text in zip(kmeans_neg.labels_, neg):
    clusters.append({'sentiment':'negative','cluster':label,'review':text})
for label, text in zip(kmeans_pos.labels_, pos):
    clusters.append({'sentiment':'positive','cluster':label,'review':text})
res_df = pd.DataFrame(clusters)

# 6) 클러스터별 대표 리뷰 3개씩 뽑아 보기
for sentiment in ['negative','positive']:
    print(f"=== {sentiment.upper()} ===")
    for c in sorted(res_df[res_df.sentiment==sentiment].cluster.unique()):
        print(f"-- Cluster {c}:")
        for r in res_df[(res_df.sentiment==sentiment)&(res_df.cluster==c)]['review'].head(3):
            print(" •", r)
        print()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.timm_wrapper.configuration_timm_wrapper because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [44]:
df = pd.read_csv(file_path,encoding='cp949',engine='python')
df.head()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand
0,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림처럼 부드럽게 피부에 스며들어 촉촉히게 발리는 라네즈 썬크림,원지,28000.0,23800.0,491.0,라네즈
1,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,"크림같아서 좋네요선크림 특유의 질감, 냄새도 없고백탁도없고 그냥 크림같아요",원지,28000.0,23800.0,491.0,라네즈
2,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림을 바르는 듯한 발림성에 촉촉함까지!! 오랜만에 정말 만족스러운 썬크림이에요,원지,28000.0,23800.0,491.0,라네즈
3,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,백탁도 없고 발림성. 흡수력 너무 좋아요. 부담없는 선크림. 수분크림 같아서 아주 ...,원지,28000.0,23800.0,491.0,라네즈
4,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,아 원지 역시 완벽주의자 원지가 만든 건 다 좋아!완전 로션제형에 슥슥 잘 발리고 ...,원지,28000.0,23800.0,491.0,라네즈


In [21]:
df = df.drop_duplicates(['review'])
df = df.reset_index(drop=True)
df.head()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand
0,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림처럼 부드럽게 피부에 스며들어 촉촉히게 발리는 라네즈 썬크림,원지,28000.0,23800.0,491.0,라네즈
1,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,"크림같아서 좋네요선크림 특유의 질감, 냄새도 없고백탁도없고 그냥 크림같아요",원지,28000.0,23800.0,491.0,라네즈
2,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림을 바르는 듯한 발림성에 촉촉함까지!! 오랜만에 정말 만족스러운 썬크림이에요,원지,28000.0,23800.0,491.0,라네즈
3,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,백탁도 없고 발림성. 흡수력 너무 좋아요. 부담없는 선크림. 수분크림 같아서 아주 ...,원지,28000.0,23800.0,491.0,라네즈
4,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,아 원지 역시 완벽주의자 원지가 만든 건 다 좋아!완전 로션제형에 슥슥 잘 발리고 ...,원지,28000.0,23800.0,491.0,라네즈


In [25]:
import re

df['review'] = df['review'].astype(str)

def extract_word(text): #한글만 남기고 삭제
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result

df['review'] = df['review'].apply(lambda x:extract_word(x))
df['tokens'] = df['review'].apply(lambda x: re.sub(r' ', r'',x))

null_list = [] #한글만 남기면 전부 지워지는 리뷰 데이터 삭제
for i in range(len(df)):
    if len(df['tokens'][i])==0:
        null_list.append(i)

for idx in null_list:
    df = df.drop(index=idx, axis=0)

df = df.reset_index(drop=True)

In [26]:
df=df.drop(index=0, axis=0) #0번째 행 삭제
df=df.reset_index(drop=True)

In [27]:
df.head()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand,tokens
0,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,크림같아서 좋네요선크림 특유의 질감 냄새도 없고백탁도없고 그냥 크림같아요,원지,28000.0,23800.0,491.0,라네즈,크림같아서좋네요선크림특유의질감냄새도없고백탁도없고그냥크림같아요
1,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림을 바르는 듯한 발림성에 촉촉함까지 오랜만에 정말 만족스러운 썬크림이에요,원지,28000.0,23800.0,491.0,라네즈,수분크림을바르는듯한발림성에촉촉함까지오랜만에정말만족스러운썬크림이에요
2,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,백탁도 없고 발림성 흡수력 너무 좋아요 부담없는 선크림 수분크림 같아서 아주 ...,원지,28000.0,23800.0,491.0,라네즈,백탁도없고발림성흡수력너무좋아요부담없는선크림수분크림같아서아주잘사용하고있어요
3,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,아 원지 역시 완벽주의자 원지가 만든 건 다 좋아 완전 로션제형에 슥슥 잘 발리고 ...,원지,28000.0,23800.0,491.0,라네즈,아원지역시완벽주의자원지가만든건다좋아완전로션제형에슥슥잘발리고화장도잘먹는데미쳐버러요
4,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,원지씨 애정하는데 유튭 보다가 바로 구매해버렸네용 일단 밴드가 넘 예쁘고 ...,원지,28000.0,23800.0,491.0,라네즈,원지씨애정하는데유튭보다가바로구매해버렸네용일단밴드가넘예쁘고선크림끈적임없고건조함도덜하고...


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27905 entries, 0 to 27904
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  27905 non-null  object 
 1   rating        27905 non-null  object 
 2   skin_type     0 non-null      float64
 3   review        27905 non-null  object 
 4   model         96 non-null     object 
 5   price1        291 non-null    float64
 6   price2        291 non-null    float64
 7   total_review  291 non-null    float64
 8   brand         291 non-null    object 
 9   tokens        27905 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.1+ MB


In [31]:
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
from matplotlib import pyplot as plt

In [32]:
from collections import Counter, defaultdict

In [35]:
from tqdm.notebook import tqdm

tqdm.pandas()
df['tokens'] = df['tokens'].progress_apply(lambda x: eval(x))

  0%|          | 0/27905 [00:00<?, ?it/s]

NameError: name '크림같아서좋네요선크림특유의질감냄새도없고백탁도없고그냥크림같아요' is not defined

In [55]:
df = pd.read_csv(file_path,encoding='cp949',engine='python')

# 브랜드 추출 함수
def extract_brand(name: str) -> str:
    """
    product_name에서 ']' 뒤 첫 번째 공백 전까지의 문자열을 반환.
    실패 시 None 반환.
    """
    m = re.search(r'\]\s*[^가-힣]*([가-힣]+)', str(name))
    return m.group(1) if m else None

df['brand'] = df['product_name'].apply(extract_brand)

# 결과 확인
df.tail()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand
34705,[5월 올영픽/파데프리] 라로슈포제 유비데아 톤업 로지 글로우 30ml 기획 (+유...,5점만점에 5점,NaN,가족선물로 시켰어요.순하고 트러블 없고 톤업돼서 이 선크림만 사용중입니다,NaN,NaN,NaN,NaN,라로슈포제
34706,[5월 올영픽/파데프리] 라로슈포제 유비데아 톤업 로지 글로우 30ml 기획 (+유...,5점만점에 5점,NaN,몇년째 사용중인 선크림이에요.자극없이순하고 발림성도 좋고 톤업돼서 하루종일 화사해요,NaN,NaN,NaN,NaN,라로슈포제
34707,[5월 올영픽/파데프리] 라로슈포제 유비데아 톤업 로지 글로우 30ml 기획 (+유...,5점만점에 5점,NaN,이거 너무너무좋아요!! 톤업도 잘되고 결정적으로 색상이 너무 이뻐용,NaN,NaN,NaN,NaN,라로슈포제
34708,[신제품/워터프루프] 비레디 하이드로 스웻프루프 선크림 50ml,5점만점에 5점,NaN,촉촉한데 바르면 보송하게 마무리 됩니다.야외 활동이 많아진 무더운 여름몸 전체에 바...,NaN,NaN,NaN,NaN,비레디
34709,[신제품/워터프루프] 비레디 하이드로 스웻프루프 선크림 50ml,5점만점에 5점,NaN,백탁없다고 해도 은근히 백탁있는 제품이 많던데이건 진짜 백탁없어요.일주일동안 사용했...,NaN,NaN,NaN,NaN,비레디


In [56]:
df.head()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand
0,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림처럼 부드럽게 피부에 스며들어 촉촉히게 발리는 라네즈 썬크림,원지,28000.0,23800.0,491.0,라네즈
1,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,"크림같아서 좋네요선크림 특유의 질감, 냄새도 없고백탁도없고 그냥 크림같아요",원지,28000.0,23800.0,491.0,라네즈
2,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,수분크림을 바르는 듯한 발림성에 촉촉함까지!! 오랜만에 정말 만족스러운 썬크림이에요,원지,28000.0,23800.0,491.0,라네즈
3,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,백탁도 없고 발림성. 흡수력 너무 좋아요. 부담없는 선크림. 수분크림 같아서 아주 ...,원지,28000.0,23800.0,491.0,라네즈
4,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5점만점에 5점,NaN,아 원지 역시 완벽주의자 원지가 만든 건 다 좋아!완전 로션제형에 슥슥 잘 발리고 ...,원지,28000.0,23800.0,491.0,라네즈


In [58]:
import numpy as np

# 2) 원본 rating 샘플 몇 개 미리 확인해 보기
print(df['rating'].dropna().unique()[:10])

def parse_rating_numbers(s):
    nums = pd.Series(str(s)).str.findall(r'(\d+)').iloc[0]
    if nums:
        return int(nums[-1])  # 마지막 숫자
    else:
        return np.nan

df['rating_num'] = df['rating'].apply(parse_rating_numbers)


# 4) 매칭되지 않은(=NaN) 행 개수 확인
print("매칭 실패 행 수:", df['rating_num'].isna().sum(), "/", len(df))

# 5) 평점이 없는 행(극소수일 가능성)을 제거하고 정수로 변환
df = df.dropna(subset=['rating_num']).reset_index(drop=True)
df['rating'] = df['rating_num'].astype(int)
df = df.drop(columns=['rating_num'])

print("최종 남은 행 수:", len(df))
print(df['rating'].value_counts())


[5 3 4 1 2]
매칭 실패 행 수: 0 / 34710
최종 남은 행 수: 34710
rating
5    29190
4     3680
3     1157
1      358
2      325
Name: count, dtype: int64


In [59]:
df.head()

,product_name,rating,skin_type,review,model,price1,price2,total_review,brand
0,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5,NaN,수분크림처럼 부드럽게 피부에 스며들어 촉촉히게 발리는 라네즈 썬크림,원지,28000.0,23800.0,491.0,라네즈
1,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5,NaN,"크림같아서 좋네요선크림 특유의 질감, 냄새도 없고백탁도없고 그냥 크림같아요",원지,28000.0,23800.0,491.0,라네즈
2,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5,NaN,수분크림을 바르는 듯한 발림성에 촉촉함까지!! 오랜만에 정말 만족스러운 썬크림이에요,원지,28000.0,23800.0,491.0,라네즈
3,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5,NaN,백탁도 없고 발림성. 흡수력 너무 좋아요. 부담없는 선크림. 수분크림 같아서 아주 ...,원지,28000.0,23800.0,491.0,라네즈
4,[품절대란/원지선크림] NEW 라네즈 워터뱅크 UV 베리어 선크림,5,NaN,아 원지 역시 완벽주의자 원지가 만든 건 다 좋아!완전 로션제형에 슥슥 잘 발리고 ...,원지,28000.0,23800.0,491.0,라네즈


In [63]:
df['brand'].unique()

array(['라네즈', '조선미녀', '구달', '라운드랩', '용량증정', '셀퓨전씨', '마스터즈', '닥터지', '식물나라',
       '넘버즈인', '메이크프렘', '아누아', '라로슈포제', '스킨', None, '체이싱래빗', '프롬리에',
       '아이레놀', '제로이드', '듀이트리', '비플레인', '메디큐브', '메디힐', '부쉬맨', '그린핑거',
       '차앤박', '딘시', '에뛰드', '토리든', '아이오페', '아떼', '프리메라', '코스알엑스', '에딧비',
       '헤라', '달바', '아넷사', '싸이닉', '이니스프리', '에스쁘아', '바이오힐', '이즈앤트리', '쏘내추럴',
       '에스네이처', '아비브', '로벡틴', '세이프온', '닥터자르트', '파티온', '피지오겔', '프란츠',
       '라타플랑', '니베아', '미샤', '퍼펙트', '선크림', '샤이샤이샤이', '스킨푸드', '더하르나이',
       '라운드어라운드', '바이오더마', '토코보', '바디언스', '유이크', '포트레', '리쥬란힐러', '어뮤즈',
       '코스노리', '키엘', '휘게', '리얼베리어', '마몽드', '아벤느', '더샘', '웰라쥬', '브링그린',
       '홀리카홀리카', '벤튼', '프리미엄', '닥터올가', '애즈이즈투비', '그린프로폴리스', '닥터디퍼런트',
       '클레어스', '진정선크림', '아크네스', '비레디', '성분에디터', '이니프리', '하멜', '블리브유',
       '누드', '메디필', '브이앤에이뷰티', '바이오던스', '키스', '메디톡스', '아이레시피', '믹순', '히야',
       '써니콘', '가히', '파이코어', '톤핏선', '알리', '에스트라', '네오젠', '유세린', '글루타치온',
       '비긴스', '바닐라코', '리쥬란', '톡스앤필'], dtype=object)

In [64]:
df['brand'].value_counts()

,count
brand,
셀퓨전씨,1124
달바,1084
피지오겔,730
스킨,726
라운드랩,710
...,...
메디톡스,7
진정선크림,7
미샤,5


In [65]:
df.to_csv("modified_reviews.csv", index=False, encoding = "utf-8-sig")

# 2) Colab 세션에서 바로 다운로드
from google.colab import files
files.download("modified_reviews.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>